# LIN 313 Final Project
_**By Jed Wang**_

In [ ]:
# if True, complete tokenizer; if False, verified tokenizer
complete = False

This is purely a utility notebook for tokenizing datasets.

In [ ]:
!pip install transformers datasets tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.5 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import login
login()

^ Login here

^ Login here

In [ ]:
# !rm tokenizer_full.json
!wget -O tokenizer_full.json https://gist.github.com/LeftistTachyon/4191cfccf4ea81277bb1976f59e7341e/raw/e2ff88b0423a12be257b5c3418f9a941392d4de4/tokenizer_full.json

--2023-11-27 17:14:06--  https://gist.github.com/LeftistTachyon/4191cfccf4ea81277bb1976f59e7341e/raw/e2ff88b0423a12be257b5c3418f9a941392d4de4/tokenizer_full.json
Resolving gist.github.com (gist.github.com)... 140.82.113.3
Connecting to gist.github.com (gist.github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/LeftistTachyon/4191cfccf4ea81277bb1976f59e7341e/raw/e2ff88b0423a12be257b5c3418f9a941392d4de4/tokenizer_full.json [following]
--2023-11-27 17:14:06--  https://gist.githubusercontent.com/LeftistTachyon/4191cfccf4ea81277bb1976f59e7341e/raw/e2ff88b0423a12be257b5c3418f9a941392d4de4/tokenizer_full.json
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6471221 (

In [ ]:
# !rm tokenizer_veri.json
!wget -O tokenizer_veri.json https://gist.github.com/LeftistTachyon/4191cfccf4ea81277bb1976f59e7341e/raw/e2ff88b0423a12be257b5c3418f9a941392d4de4/tokenizer_veri.json

--2023-11-27 17:14:06--  https://gist.github.com/LeftistTachyon/4191cfccf4ea81277bb1976f59e7341e/raw/e2ff88b0423a12be257b5c3418f9a941392d4de4/tokenizer_veri.json
Resolving gist.github.com (gist.github.com)... 140.82.113.3
Connecting to gist.github.com (gist.github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/LeftistTachyon/4191cfccf4ea81277bb1976f59e7341e/raw/e2ff88b0423a12be257b5c3418f9a941392d4de4/tokenizer_veri.json [following]
--2023-11-27 17:14:06--  https://gist.githubusercontent.com/LeftistTachyon/4191cfccf4ea81277bb1976f59e7341e/raw/e2ff88b0423a12be257b5c3418f9a941392d4de4/tokenizer_veri.json
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6196696 (

In [ ]:
from transformers import PreTrainedTokenizerFast

# Instantiate the tokenizer from before
tokenizer = PreTrainedTokenizerFast(
    tokenizer_file="tokenizer_full.json" if complete else "tokenizer_veri.json",
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    mask_token="[MASK]",
    model_max_length=512
)

## Actually tokenizing the dataset

In [ ]:
# the dataset we're going to tokenize today
dataset_name = "C-MTEB/AFQMC"

cust_dataset_name = dataset_name[dataset_name.find("/")+1:]
dataset_name, cust_dataset_name

('C-MTEB/AFQMC', 'AFQMC')

In [ ]:
from datasets import load_dataset

dataset = load_dataset(dataset_name, num_proc=4)

Setting num_proc from 4 back to 1 for the test split to disable multiprocessing as it only contains one shard.


Generating test split:   0%|          | 0/3861 [00:00<?, ? examples/s]

Setting num_proc from 4 back to 1 for the train split to disable multiprocessing as it only contains one shard.


Generating train split:   0%|          | 0/34334 [00:00<?, ? examples/s]

Setting num_proc from 4 back to 1 for the validation split to disable multiprocessing as it only contains one shard.


Generating validation split:   0%|          | 0/4316 [00:00<?, ? examples/s]

In [ ]:
import multiprocessing

def tokenize_function(examples):
  return tokenizer(examples["sentence1"], examples["sentence2"], padding="max_length", truncation=True)

tokenized_dataset = dataset.map(tokenize_function,
                                batched=True,
                                num_proc=multiprocessing.cpu_count())

Map (num_proc=8):   0%|          | 0/3861 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/34334 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/4316 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset.push_to_hub(
    repo_id=f"jedwang/{cust_dataset_name}",
    # revision=dataset_name,
    commit_message=f"Completed pretokenization of {dataset_name}!",
    private=True
)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/35 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

In [ ]:
load_dataset(f"jedwang/{cust_dataset_name}", num_proc=4)

Setting num_proc from 4 back to 1 for the test split to disable multiprocessing as it only contains one shard.


Generating test split:   0%|          | 0/3861 [00:00<?, ? examples/s]

Setting num_proc from 4 back to 1 for the train split to disable multiprocessing as it only contains one shard.


Generating train split:   0%|          | 0/34334 [00:00<?, ? examples/s]

Setting num_proc from 4 back to 1 for the validation split to disable multiprocessing as it only contains one shard.


Generating validation split:   0%|          | 0/4316 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['sentence1', 'sentence2', 'score', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3861
    })
    train: Dataset({
        features: ['sentence1', 'sentence2', 'score', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 34334
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'score', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4316
    })
})